In [2]:
import pandas as pd

In [4]:
pd.DataFrame??

In [1]:
from __future__ import division
import pandas as pd
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

rcParams['figure.figsize'] = 20, 5
warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")

from scipy.sparse import csr_matrix
from zipcode_mapping import zipcode_mapping

In [2]:
df = pd.read_pickle('../data/data_w_transformed_census_and_removed_invalid_rows_and_cols_and_fixed_zips_and_descs_and_dropped_latlongs.pkl')
df.shape

(744372, 37)

## Preprocessing

In [3]:
df.Property_Type = df.Property_Type.fillna('other')
df.Property_ID = df.Property_ID.fillna('other')

In [ ]:
df.head(1).T

In [ ]:
old_df = df.copy()

In [ ]:
old_df.shape

In [4]:
df = df[['TYPE', 'REASON', 'Department', 'SubmittedPhoto', 'Property_Type', \
    'Source', 'neighborhood_from_zip', 'zipcode', 'COMPLETION_TIME', 'race_white', 'race_black', \
    'race_asian', 'race_hispanic', 'race_other', 'poverty_pop_below_poverty_level', \
    'poverty_pop_w_public_assistance', 'poverty_pop_w_food_stamps', 'poverty_pop_w_ssi', \
    'school', 'housing', 'bedroom', 'value', 'rent', 'income']]
df.shape

(744372, 24)

In [5]:
df.isnull().sum()

TYPE                                   0
REASON                                 0
Department                             0
SubmittedPhoto                         0
Property_Type                          0
Source                                 0
neighborhood_from_zip                  0
zipcode                                0
COMPLETION_TIME                    65765
race_white                             0
race_black                             0
race_asian                             0
race_hispanic                          0
race_other                             0
poverty_pop_below_poverty_level        0
poverty_pop_w_public_assistance        0
poverty_pop_w_food_stamps              0
poverty_pop_w_ssi                      0
school                                 0
housing                                0
bedroom                                0
value                                  0
rent                                   0
income                                 0
dtype: int64

In [6]:
df = df.dropna()

## Dummifying

In [7]:
def dummify(df, column):
    # from Darren's linear regression slides
    print '{} is your baseline'.format(sorted(df[column].unique())[-1])
    dummy = pd.get_dummies(df[column]).rename(columns=lambda x: column+'_'+str(x)).iloc[:,0:len(df[column].unique())-1]
    df = df.drop(column,axis=1) #Why not inplace? because if we do inplace, it will affect the df directly
    return pd.concat([df,dummy],axis=1)

In [8]:
df1 = dummify(df, 'TYPE')
df2 = dummify(df1, 'REASON')
df3 = dummify(df2, 'Department')
df4 = dummify(df3, 'Property_Type')
df5 = dummify(df4, 'Source')
df6 = dummify(df5, 'neighborhood_from_zip')
df7 = dummify(df6, 'zipcode')
df8 = dummify(df7, 'school')
df9 = dummify(df8, 'housing')

Zoning is your baseline
Weights and Measures is your baseline
PWDx is your baseline
other is your baseline
Twitter is your baseline
West Roxbury is your baseline
2467.0 is your baseline
8_6th_grade is your baseline
rent is your baseline


MemoryError: 

In [ ]:
df9.shape

## Running model

Let's try w/o GridSearchCV; is it faster?

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


Splitting train/test 80/20:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df7.drop('COMPLETION_TIME', axis=1), 
    df7.COMPLETION_TIME, 
    test_size=0.2, 
    random_state=300
)